# Visualizing scikit-learn pipelines in Jupyter

The goal of keeping this notebook is to:

- make it available for users that want to reproduce it locally
- archive the script in the event we want to rerecord this video with an
  update in the UI of scikit-learn in a future release.

## First we load the dataset

We need to define our data and target. In this case we will build a classification model

In [1]:
import pandas as pd

ames_housing = pd.read_csv("../../datasets/house_prices.csv", na_values='?')

target_name = "SalePrice"
data, target = ames_housing.drop(columns=target_name), ames_housing[target_name]
target = (target > 200_000).astype(int)

In [8]:
ames_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

We inspect the first rows of the dataframe

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LotArea       1460 non-null   int64 
 1   FullBath      1460 non-null   int64 
 2   HalfBath      1460 non-null   int64 
 3   Neighborhood  1460 non-null   object
 4   HouseStyle    1460 non-null   object
dtypes: int64(3), object(2)
memory usage: 57.2+ KB


For the sake of simplicity, we can cherry-pick some features and only retain
this arbitrary subset of data:

In [3]:
numeric_features = ['LotArea', 'FullBath', 'HalfBath']
categorical_features = ['Neighborhood', 'HouseStyle']
data = data[numeric_features + categorical_features]

## Then we create the pipeline

The first step is to define the preprocessing steps

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(),
)])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

The next step is to apply the transformations using `ColumnTransformer`

In [7]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
])

Then we define the model and join the steps in order

In [9]:
from sklearn.linear_model import LogisticRegression

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression()),
])

Let's visualize it!

In [10]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['LotArea', 'FullBath',
                                                   'HalfBath']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Neighborhood',
                                                   'HouseStyle'])])),
                ('classifier', LogisticRegression())])

## Finally we score the model

In [11]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")

The mean cross-validation accuracy is: 0.859 ± 0.018


<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p>In this case, around 86% of the times the pipeline correctly predicts whether
the price of a house is above or below the 200_000 dollars threshold. But
be aware that this score was obtained by picking some features by hand, which
is not necessarily the best thing we can do for this classification task. In this
example we can hope that fitting a complex machine learning pipelines on a
richer set of features can improve upon this performance level.</p>
<p class="last">Reducing a price estimation problem to a binary classification problem with a
single threshold at 200_000 dollars is probably too coarse to be useful in
in practice. Treating this problem as a regression problem is probably a better
idea. We will see later how to train and evaluate the performance
of various regression models.</p>
</div>